# Deploy MLflow Model to online endpoint

- Before running this notebook, run the **Chapter 6 Prep-Model Creation & Registration.ipynb** notebook to create and register a model for use

- Models created with MLFlow do not require a scoring script nor an environment


In [6]:
import azure.ai.ml
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)

print(azure.ai.ml._version.VERSION)

0.1.0b6


In [7]:
subscription_id = '5da07161-3770-4a4b-aa43-418cbbb627cf'
resource_group = 'aml-workspace-rg'
workspace = 'aml-workspace'

In [8]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

In [9]:
#connect to the workspace
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": subscription_id,
        "resource_group": resource_group,
        "workspace_name": workspace,
    }

    if client_config["subscription_id"].startswith("<"):
        print(
            "please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <AML_WORKSPACE_NAME> in notebook cell"
        )
        raise ex
    else:  # write and reload from config file
        import json, os

        config_path = "../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz1/code/Users/memasanz/.azureml/config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7fef36462f70>,
         subscription_id=5da07161-3770-4a4b-aa43-418cbbb627cf,
         resource_group_name=aml-workspace-rg,
         workspace_name=aml-workspace)


## Create Online Endpoint

To create an online endpoint, we will leverage the class *ManagedOnlineEndpoint*.  To create the online endpoint we will provide the following configuration:

- name of endpoint
- description
- auth_mode (set to key) or aml_token
- tags - to provide information regarding the endpoint

### Configure endpoint

In [10]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="titanic online endpoint for mlflow model",
    auth_mode="key",
    tags={"mlflow": "true"},
)

## Create endpoint

Using the MLClient created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.


In [11]:
ml_client.begin_create_or_update(endpoint)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-08272043345225.eastus.inference.ml.azure.com/score', 'swagger_uri': 'https://endpoint-08272043345225.eastus.inference.ml.azure.com/swagger.json', 'name': 'endpoint-08272043345225', 'description': 'titanic online endpoint for mlflow model', 'tags': {'mlflow': 'true'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resourcegroups/aml-workspace-rg/providers/microsoft.machinelearningservices/workspaces/aml-workspace/onlineendpoints/endpoint-08272043345225', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:88449c92-2fa7-4734-8cc5-76e1776b55bb:a97ac231-6bf4-42ff-aed9-fac788487939?api-version=2022-02-01-preview'}, 'id': '/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resource

## Create deployment
A deployment is a set of resouces used for hosting the inferecing model using the *ManagedOnlineDeployment* class.  
Using the *ManagedOnlineDeployment* class, a developer can configure the following components

- name: name of the deployment
- endpoint_name: name of the endpoint to create the deployment under
- model: the model to use for the deployment
- instance_type: the VM side to use for deployment
- instance_count: the number of instances to use for the deployment
    

# Retrieve Model from registry

In [58]:
import mlflow
experiment = 'Chapter6'
current_experiment=dict(mlflow.get_experiment_by_name(experiment))
experiment_id=current_experiment['experiment_id']
print(experiment_id)

df = mlflow.search_runs([experiment_id])


run_id = df['run_id'].tail(1)
print(run_id)

7685ed2a-881e-4e77-8c82-429623276f43
1    mighty_glass_c9tc069gfn
Name: run_id, dtype: object


In [82]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

run_model = Model(
    name="chapter6_titanic_model",
    path="azureml://jobs/coral_gas_1hz6njncjl/outputs/artifacts/paths/model/",
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL 
)


In [83]:
run_model

Model({'job_name': None, 'is_anonymous': False, 'auto_increment_version': True, 'name': 'chapter6_titanic_model', 'description': 'Model created from run.', 'tags': {}, 'properties': {}, 'id': None, 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz1/code/Users/memasanz/ML-Engineering-with-Azure-Machine-Learning-Service/Chapter6', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fef15eb8400>, 'version': None, 'latest_version': None, 'path': 'azureml://jobs/coral_gas_1hz6njncjl/outputs/artifacts/paths/model/', 'utc_time_created': None, 'flavors': None, 'arm_type': 'model_version', 'type': 'mlflow_model'})

## Configure the deployment

- retrieve the experiment id for this run, and the run id to retrieve the model from the registered model list

In [84]:

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=run_model,
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

In [85]:
## Create deployment

In [86]:
ml_client.online_deployments.begin_create_or_update(blue_deployment)

Check: endpoint endpoint-08272043345225 exists
Creating/updating online deployment blue 

..................................................................................................

Done (8m 38s)


In [87]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-08272043345225.eastus.inference.ml.azure.com/score', 'swagger_uri': 'https://endpoint-08272043345225.eastus.inference.ml.azure.com/swagger.json', 'name': 'endpoint-08272043345225', 'description': 'titanic online endpoint for mlflow model', 'tags': {'mlflow': 'true'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resourcegroups/aml-workspace-rg/providers/microsoft.machinelearningservices/workspaces/aml-workspace/onlineendpoints/endpoint-08272043345225', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:88449c92-2fa7-4734-8cc5-76e1776b55bb:969edb9f-7d8f-4f00-b1dd-0f06ff80272a?api-version=2022-02-01-preview'}, 'id': '/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resource

In [88]:
## Get Endpoint details

# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(endpoint)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

primary_key = ml_client.online_endpoints.list_keys(name = online_endpoint_name).primary_key

print(primary_key)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-08272043345225.eastus.inference.ml.azure.com/score', 'swagger_uri': 'https://endpoint-08272043345225.eastus.inference.ml.azure.com/swagger.json', 'name': 'endpoint-08272043345225', 'description': 'titanic online endpoint for mlflow model', 'tags': {'mlflow': 'true'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resourcegroups/aml-workspace-rg/providers/microsoft.machinelearningservices/workspaces/aml-workspace/onlineendpoints/endpoint-08272043345225', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:88449c92-2fa7-4734-8cc5-76e1776b55bb:969edb9f-7d8f-4f00-b1dd-0f06ff80272a?api-version=2022-02-01-preview'}, 'id': '/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resource

In [89]:
ml_client.online_endpoints.list_keys(name = online_endpoint_name).primary_key

'oNT7EGxQiIYR6i9czXAhZsF8G4SUzB5K'

In [90]:
from azureml.core import Workspace, Dataset

ws = Workspace(subscription_id, resource_group, workspace)

dataset = Dataset.get_by_name(ws, name='Titanic-tabular-dataset')
df = dataset.to_pandas_dataframe()

columns_to_keep =  ['Embarked', 'Loc', 'Sex','Pclass', 'Age', 'Fare', 'GroupSize']
X_raw           = df[columns_to_keep]
X_raw

,Embarked,Loc,Sex,Pclass,Age,Fare,GroupSize
0,S,X,male,3,22.0,7.2500,2
1,C,C,female,1,38.0,71.2833,2
2,S,X,female,3,26.0,7.9250,1
3,S,C,female,1,35.0,53.1000,2
4,S,X,male,3,35.0,8.0500,1
...,...,...,...,...,...,...,...
886,S,X,male,2,27.0,13.0000,1
887,S,B,female,1,19.0,30.0000,1
888,S,X,female,3,21.5,23.4500,4
889,C,C,male,1,26.0,30.0000,1


In [91]:
import json
url = endpoint.scoring_uri
api_key = primary_key  # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'blue' }
import requests

def MakePrediction(df):
    strjson = str(df.to_json(orient='records'))
    endpoint_url = url

    request_data =  {
              "input_data": {
                "columns": [
                  "Embarked",
                  "Loc",
                  "Sex",
                  "Pclass",
                  "Age",
                  "Fare",
                  "GroupSize"
                ],
                "data": []
              }
            }

    request_data['input_data']['data'] = json.loads(X_raw.to_json(orient='records'))
    parsed = json.dumps(request_data)
    
    r = requests.post(endpoint_url, headers=headers, data=parsed)
    return (r.json())

results = MakePrediction(X_raw)
print(results)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 